In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: Example_OP_Manish.xlsx, id: 1--fNGDIgLj6aLDr0cWacpB_ToaxmDjl9
title: Example_OP.xlsx, id: 1pEm_sAvr2BFf2g8KZESLG5RYH1I8Glk2
title: Classroom, id: 0B_gPr3BosfdmflRjdUNJNlhjTGlpdEVjX0U3SWxlY1BaZko1QnE3MHFWSE5wX3VGTTN2aTA
title: DDOS_2019_Sample_File.csv, id: 1BDMPjvS-5yBi7XZve2maVnFMsQWuoa-7
title: DDOS_Train - Copy.xlsx, id: 12AH-hKoMZgQDEvV1fxJq-j58kpA46MF0
title: DDOS_Train.xlsx, id: 17pgNoCZ7BtSYc1wIypBDSkiJNbIUSSdx
title: DDOS_Test.csv, id: 1sKD6QuBVyaEDjPb-mCDkPjwfAd8-YaDm
title: NSL_KDD_Train.csv, id: 1NdNFFeHHC7_wbGl67owDY340O-DzwAOn
title: DrDoS_NTP_Train.csv, id: 1UtZFETT1ldMj2FRvq6vkDWlzkxxnhJ5s
title: Portmap-train - 10.csv, id: 1nvW3yWfVj3Sm8KlT069w0TtbMvzDu9x2
title: Colab Notebooks, id: 1-gHxYdhAkhUseRaT3gzWT4MPxPDP8fmX
title: RM1 Submission, id: 1ulNMgBVt790mNJ3D5j79ppRmQ_i35rBV
title: Hash and sc, id: 1P-3Ta-F1e7MzcbtW0zGM1C0I-Z9eDh3G
title: Ecommerce Project SS, id: 1LLQZAzE4m59vlgzsy_Wkx37LPSAJVRYY
title: Manish Patel_New-CV.docx, id: 1I6uOjjOtvhwDWTbNiQf0y6Z2pA

In [3]:
dataset = drive.CreateFile({'id': '1BDMPjvS-5yBi7XZve2maVnFMsQWuoa-7'})
dataset.GetContentFile('dataset')
dataframe=pd.read_csv('dataset')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (85) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
dataframe = dataframe.drop(['Flow ID'], axis=1)
dataframe = dataframe.drop(['Unnamed: 0'], axis=1)
dataframe = dataframe.drop(['Timestamp'], axis=1)


nonImportantColumns = ['Source IP','Destination IP','Protocol','Flow Duration','Total Fwd Packets','Total Backward Packets','Fwd Packet Length Std','Bwd Packet Length Std',
                       'Flow Bytes/s','Flow Packets/s','Flow IAT Max','Fwd IAT Total','Fwd IAT Max','Bwd IAT Total','Bwd IAT Std','Bwd IAT Max','Bwd IAT Min','Fwd PSH Flags',
                        'Bwd PSH Flags','Fwd URG Flags','Bwd URG Flags','Fwd Header Length','Bwd Header Length','Fwd Packets/s','Packet Length Std','FIN Flag Count','SYN Flag Count',
                        'RST Flag Count','PSH Flag Count','ACK Flag Count','URG Flag Count','CWE Flag Count','ECE Flag Count','Down/Up Ratio','Fwd Header Length.1','Fwd Avg Bytes/Bulk',
                        'Fwd Avg Packets/Bulk','Fwd Avg Bulk Rate','Bwd Avg Bytes/Bulk','Bwd Avg Packets/Bulk','Bwd Avg Bulk Rate','Subflow Fwd Packets','Subflow Bwd Packets',
                        'act_data_pkt_fwd','min_seg_size_forward','Active Std','Active Max','Active Min','Idle Mean','Idle Std','Idle Max','Idle Min','SimillarHTTP','Inbound']
for column in nonImportantColumns :
 dataframe = dataframe.drop([column], axis=1)

dataframe.columns = dataframe.columns.str.replace(' ', '_')

In [5]:
dataframe = dataframe.replace([np.inf, -np.inf], np.nan)
dataframe = dataframe.dropna()


In [6]:
dataframe['Label'].unique()

array(['BENIGN', 'DrDoS_DNS', 'DrDoS_LDAP', 'DrDoS_MSSQL',
       'DrDoS_NetBIOS', 'DrDoS_NTP', 'DrDoS_SSDP', 'DrDoS_UDP', 'Syn',
       'TFTP', 'WebDDoS', 'UDP-lag', 'UDP', 'UDPLag', 'MSSQL', 'Portmap',
       'NetBIOS', 'LDAP'], dtype=object)

In [7]:
dataframe['Label']=dataframe['Label'].map( {'BENIGN':0, 'DrDoS_DNS':1, 'DrDoS_LDAP':1, 'DrDoS_MSSQL':1, 'DrDoS_NetBIOS':1, 'DrDoS_NTP':1, 'LDAP':1, 
                                            'DrDoS_SSDP':1, 'DrDoS_UDP':1, 'NetBIOS':1, 'Portmap':1, 'Syn':1, 'TFTP':1, 'UDP':1, 'UDPLag':1, 'UDP-lag':1, 'WebDDoS':1, 'MSSQL':1,   } )

In [8]:
dataframe

Source_Port  Destination_Port  ...   Active_Mean  Label
0             59099                80  ...  198805.45450      0
1             59102               443  ...   90226.50000      0
2                 0                 0  ...  248176.75000      0
3             59155                80  ...   14901.22222      0
4             59131                80  ...   26508.33333      0
...             ...               ...  ...           ...    ...
141953        60159               443  ...       0.00000      0
141954        60160               443  ...       0.00000      0
141955        60156               443  ...       0.00000      0
141956        60162               443  ...       0.00000      0
141957        60161               443  ...       0.00000      0

[141958 rows x 31 columns]

In [9]:
is_1 =  dataframe["Label"]==1
allone = dataframe[is_1]
print(allone.shape)

is_0 =  dataframe["Label"]==0
allzero = dataframe[is_0]
print(allzero.shape)

(71081, 31)
(70877, 31)


In [10]:
train, test = train_test_split(dataframe, test_size=0.2)

train.reset_index(drop=True,inplace=True)
test.reset_index(drop=True,inplace=True)

In [11]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Label')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [12]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [13]:
# Get Feature columns
featureColumns = []
for col in dataframe.columns: 
    featureColumns.append(col) 

feature_columns = []
for feature_name in featureColumns:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float64))
feature_columns.pop()

# A layer that produces a dense Tensor based on given feature_columns.
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
print(feature_columns)
"""
feature_columns1 = []
for feature_name in featureColumns:
    feature_columns1.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
"""

[NumericColumn(key='Source_Port', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None), NumericColumn(key='Destination_Port', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None), NumericColumn(key='Total_Length_of_Fwd_Packets', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None), NumericColumn(key='Total_Length_of_Bwd_Packets', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None), NumericColumn(key='Fwd_Packet_Length_Max', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None), NumericColumn(key='Fwd_Packet_Length_Min', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None), NumericColumn(key='Fwd_Packet_Length_Mean', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None), NumericColumn(key='Bwd_Packet_Length_Max', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None), NumericColumn(key='Bwd_Packet_Length_Min', shape=(1,), default_value=None, dtype=tf.float

'\nfeature_columns1 = []\nfor feature_name in featureColumns:\n    feature_columns1.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))\n'

In [14]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))

Every feature: ['Source_Port', 'Destination_Port', 'Total_Length_of_Fwd_Packets', 'Total_Length_of_Bwd_Packets', 'Fwd_Packet_Length_Max', 'Fwd_Packet_Length_Min', 'Fwd_Packet_Length_Mean', 'Bwd_Packet_Length_Max', 'Bwd_Packet_Length_Min', 'Bwd_Packet_Length_Mean', 'Flow_IAT_Mean', 'Flow_IAT_Std', 'Flow_IAT_Min', 'Fwd_IAT_Mean', 'Fwd_IAT_Std', 'Fwd_IAT_Min', 'Bwd_IAT_Mean', 'Bwd_Packets/s', 'Min_Packet_Length', 'Max_Packet_Length', 'Packet_Length_Mean', 'Packet_Length_Variance', 'Average_Packet_Size', 'Avg_Fwd_Segment_Size', 'Avg_Bwd_Segment_Size', 'Subflow_Fwd_Bytes', 'Subflow_Bwd_Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'Active_Mean']


In [ ]:
# HA Start
"""
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(64, activation='relu'),
  #don't include this layers.Dropout(.1),
  layers.Dense(1)
])
and ephochs = 2
"""
# HA End

# Feture_Layer - A layer that produces a dense Tensor based on given feature_columns .It is the first(input) layer of the model. 
# Dense layer generate an 1 output(means either 1 or else 0)

# LA Start
# Initialisation of ANN - Build neural network.
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dense(1)
])
# LA End


#Specify the Optimizer, Loss and Metrics
#If softmax layer is not being added at the last layer then we need to have the from_logits=True to indicate the probabilities are not normalized 
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

#Train data
#An epoch is an iteration over the entire x and y data provided
model.fit(train_ds, epochs=10)

Epoch 1/10
Consider rewriting this model with the Functional API.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Consider rewriting this model with the Functional API.
3549/3549 [==============================] - 9s 3ms/step - loss: 2869.4622 - accuracy: 0.9387
Epoch 2/10
3549/3549 [==============================] - 9s 3ms/step - loss: 421.5115 - accuracy: 0.9618
Epoch 3/10
3549/3549 [==============================] - 10s 3ms/step - loss: 94.0770 - accuracy: 0.9690
Epoch 4/10
3549/3549 [==============================] - 10s 3ms/step - loss: 7.2023 -

In [ ]:
# Evalute Accuracy and Loss for the testing data 
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)
print("Loss", loss)

Consider rewriting this model with the Functional API.
888/888 [==============================] - 2s 2ms/step - loss: 0.2660 - accuracy: 0.8714
Accuracy 0.8714426755905151
Loss 0.26597291231155396


In [ ]:
#Predict the values for test data
y_pred = model.predict_classes(test_ds)
print(y_pred)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Consider rewriting this model with the Functional API.
[[0]
 [0]
 [1]
 ...
 [1]
 [0]
 [1]]


In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')
cnf_matrix = metrics.confusion_matrix(test['Label'], y_pred)
class_names=[0,1]
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in
                cnf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cnf_matrix.flatten()/np.sum(cnf_matrix)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cnf_matrix, annot=labels, fmt='', cmap='Blues')


ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

TP = cnf_matrix[1, 1]
TN = cnf_matrix[0, 0]
FP = cnf_matrix[0, 1]
FN = cnf_matrix[1, 0]
sensitivity = TP / float(FN + TP)

#When the actual value is positive, how often is the prediction correct?
#When the actual value is negative, how often is the prediction correct?
specificity = TN / (TN + FP)
print('sensitivity: %0f, specificity: %0f' % (sensitivity,specificity))

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
y_pred_keras = model.predict(test_ds).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(test['Label'], y_pred)
auc_keras = auc(fpr_keras, tpr_keras)
print(auc_keras)

plt.figure()
lw = 2
plt.plot(fpr_keras, tpr_keras, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % auc_keras)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_curve
precision, recall, thresholds = precision_recall_curve(test['Label'], y_pred)
print('precision ', precision)
print('recall ', recall)
print('thresholds ', thresholds)

from sklearn.metrics import f1_score
f1 = f1_score(test['Label'], y_pred)
print('f1_score',f1)

MSE = np.square(np.subtract(test['Label'],y_pred)).mean()
print('MSE',MSE)

The sequential model allows us to create models layer-by-layer for most problems. ... It allows us to create models that have a lot more flexibility as we can easily define model with different layers.

The 5-Step Model Life-Cycle
1. Define the model.
2. Compile the model.
3. Fit the model.
4. Evaluate the model.
5. Make predictions.

**1. Define the Model**

Defining the model requires that we first select the type of model that you need and we define the other parameters.

From an API perspective, this involves defining the layers of the model, configuring each layer with a number of nodes and activation function, and connecting the layers together into a cohesive model.


**2. Compile the Model**

Compiling the model requires that you first select a loss function that you want to optimize, such as mean squared error or cross-entropy.

It also requires that you select an algorithm to perform the optimization procedure, typically stochastic gradient descent, or a modern variation, such as Adam. It may also require that you select any performance metrics to keep track of during the model training process.

From an API perspective, this involves calling a function to compile the model with the chosen configuration.

The optimizer can be specified as a string for a known optimizer class, e.g. ‘sgd‘ for stochastic gradient descent.

For a list of supported optimizers, see this: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers

For a list of supported loss functions, see:https://www.tensorflow.org/api_docs/python/tf/keras/losses

The three most common loss functions are:

‘binary_crossentropy‘ for binary classification.
‘sparse_categorical_crossentropy‘ for multi-class classification.
‘mse‘ (mean squared error) for regression.


Metrics are defined as a list of strings to evaluate predictions.
For a list of supported metrics, see:
https://www.tensorflow.org/api_docs/python/tf/keras/metrics



**3. Fit the Model**
Fitting the model requires that you first select the training configuration, such as the number of epochs (loops through the training dataset) and the batch size (number of samples in an epoch used to estimate model error).

Training applies the chosen optimization algorithm to minimize the chosen loss function and updates the model using the backpropagation of error algorithm.

Fitting the model is the slow part of the whole process and can take seconds to hours to days, depending on the complexity of the model, the hardware you’re using, and the size of the training dataset. Here, we are using Google Colab.
Colaboratory, or “Colab” for short, is a product from Google Research. Colab allows anybody to write and execute arbitrary python code through the browser, and is especially well suited to machine learning, data analysis and education.
It provides the use of free GPU (Graphics processing unit).

From an API perspective, this involves calling a function to perform the training process. This function will block (not return) until the training process has finished.

While fitting the model, a progress bar will summarize the status of each epoch and the overall training process.


**4. Evaluate the Model**

Evaluating the model requires that you first choose a holdout dataset used to evaluate the model. This should be data not used in the training process so that we can get an unbiased estimate of the performance of the model when making predictions on new data.

The speed of model evaluation is proportional to the amount of data you want to use for the evaluation, although it is much faster than training as the model is not changed.


**5. Make a Prediction**

Making a prediction is the final step in the life-cycle. It is why we wanted the model in the first place.


From an API perspective, you simply call a function to make a prediction of a class label